### Load file from a csv

In [28]:
import pandas as pd

df = pd.read_csv('order_brush_order.csv')
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


### Data cleaning

In [29]:
# Convert event_time to time object
df['event_time'] = pd.to_datetime(df.event_time)

# Add new column 'suspect' to indicate if the row is sus or not
df['suspect'] = 0

# Sort data by shopid then by userid and event_time
df = df.sort_values(by=['shopid', 'userid', 'event_time']).reset_index(drop = True)

df.head()

,orderid,shopid,userid,event_time,suspect
0,31086409141107,10009,196962305,2019-12-27 03:06:50,0
1,31144571933461,10051,2854032,2019-12-27 19:16:11,0
2,31254979546679,10051,48600461,2019-12-29 01:56:19,0
3,31431527100615,10061,62464559,2019-12-31 02:58:48,0
4,31412115824794,10061,130633421,2019-12-30 21:35:15,0


### Finding Sus row

In [30]:
# suspect = 1
# !suspect = 0
# Using the copy of the dataframe so it doesn't affect the original dataframe

df2 = df

for i, row in df2.iterrows():
    if i == 222749:
        break
    
    else:
        # compare current row with next row
        cur_time = df2['event_time'][i]
        next_time = df2['event_time'][i+1]
        cur_shopid = df2['shopid'][i]
        next_shopid = df2['shopid'][i+1]
        cur_userid = df2['userid'][i]
        next_userid = df2['userid'][i+1]
    
        sec_diff = (next_time - cur_time).total_seconds()
        
        # order within 1 hours, from same userid to same shopid is considered sus
        if ((next_shopid == cur_shopid) and (next_userid == cur_userid) and (sec_diff <= 3600)):
            df2.loc[i, 'suspect'] = 1
            df2.loc[i+1, 'suspect'] = 1

In [31]:
# drop row which userid is not suspicious 
df_sus= df2[df2.suspect != 0].reset_index(drop=True)

# drop column 'suspect'
df_sus = df_sus.drop(columns=['suspect'])

df_sus.head()

,orderid,shopid,userid,event_time
0,31242597715742,10084,180772892,2019-12-28 22:29:57
1,31242888812765,10084,180772892,2019-12-28 22:34:49
2,31409934210414,10100,211192846,2019-12-30 20:58:55
3,31410439110900,10100,211192846,2019-12-30 21:07:19
4,31418337025289,10151,31837989,2019-12-30 23:18:57


In [32]:
# calculate the total order for each shopid
no_of_order = df_sus.groupby(by='shopid', as_index=False).agg({'orderid': pd.Series.nunique})
no_of_order = no_of_order.rename(columns={"orderid": "totalOrder"})

no_of_order.head()

,shopid,totalOrder
0,10084,2
1,10100,2
2,10151,6
3,10159,13
4,10208,18


In [33]:
# calculate the total order from each userid to same shopid 
uniq_buyers = df_sus.groupby(by=['shopid', 'userid'], as_index=False).agg({'orderid': pd.Series.nunique})

uniq_buyers.head()

,shopid,userid,orderid
0,10084,180772892,2
1,10100,211192846,2
2,10151,31837989,2
3,10151,86142036,2
4,10151,91471576,2


### Calculate concentrate rate for Sus row

In [34]:
uniq_buyers['con_rate'] = 0

for i, row in uniq_buyers.iterrows():
    x = row['shopid']
    y = (no_of_order.loc[no_of_order['shopid'] == x]['totalOrder'])
    row['con_rate'] = y / row['orderid'] 

uniq_buyers.head()

,shopid,userid,orderid,con_rate
0,10084,180772892,2,1
1,10100,211192846,2,1
2,10151,31837989,2,3
3,10151,86142036,2,3
4,10151,91471576,2,3


In [35]:
# only grab rows that has concentrate value equal to or more than 3 
sus_order = uniq_buyers[uniq_buyers.con_rate >= 3].reset_index(drop=True)
sus_order.head()

,shopid,userid,orderid,con_rate
0,10151,31837989,2,3
1,10151,86142036,2,3
2,10151,91471576,2,3
3,10159,12906688,2,6
4,10159,20045254,2,6


### Grouping userid that has the same shopid with '&'

In [38]:
new_shopid=[]
new_userid=[]

temp_shopid = 0
temp_userid = []
finalize_temp_userid = ""

for i, row in sus_order.iterrows():
       
    if row['shopid'] != temp_shopid:
        temp_shopid = row['shopid']
        temp_userid.append(row['userid'])
    else:
        temp_userid.append(row['userid'])
        
    if i!= 3406:
        next = sus_order['shopid'][i+1]
    else:
        next = 0
    
    if next != row['shopid']:
        temp_userid.sort()
        for i in temp_userid:
            if i != temp_userid[-1]:
                finalize_temp_userid = finalize_temp_userid + str(i) + '&'
            else:
                finalize_temp_userid = finalize_temp_userid + str(i)
        new_shopid.append(temp_shopid)   
        new_userid.append(finalize_temp_userid)
        
        if i != 3406:
            temp_shopid = 0
            temp_userid.clear()
            finalize_temp_userid = ""
        

In [39]:
# Create new dataframe to store all sus userid and shopid
new_shopid= pd.Series(new_shopid)
new_userid= pd.Series(new_userid)
frame = {'shopid': new_shopid, 'userid': new_userid} 
new_df = pd.DataFrame(frame) 
new_df.head()

,shopid,userid
0,10151,31837989&86142036&91471576
1,10159,12906688&20045254&62775725&101114264&173897203...
2,10208,634956&2033210&11288307&29056185&32350901&5163...
3,10287,86239&1125834&8649194&58009118&74488711&103289...
4,10429,15535193&29541026&167913534


### Filter Initial dataframe from the Sus data

In [40]:
df = df[~df.shopid.isin(new_df.shopid)]
df = df.drop(columns=['orderid', 'suspect', 'event_time'])
df= df['shopid'].unique()
frame = {'shopid': df, 'userid': 0} 
df = pd.DataFrame(frame) 
df.head()

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0


### Concat Sus Dataframe with Initial Dataframe that has been filtered

In [41]:
vertical_stack = pd.concat([df, new_df], axis=0)
vertical_stack

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
420,195068512,34333529&160832423&179731889
421,198172755,127725637&129799840
422,200773571,638190&29479166&65173891&185389828&215287720
423,201503467,213506522


### Export to csv file

In [42]:
vertical_stack.to_csv("answer.csv", header = True, index=False)